## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-14-18-48-00 +0000,wsj,Wholesale prices—the cost of goods and service...,https://www.wsj.com/economy/central-banking/in...
1,2025-08-14-18-45-00 +0000,wsj,Israeli Minister Strikes at Palestinian State ...,https://www.wsj.com/world/middle-east/israeli-...
2,2025-08-14-18-44-35 +0000,nyt,Russia Seeks to Add Trade and Arms Control Int...,https://www.nytimes.com/2025/08/14/world/europ...
3,2025-08-14-18-43-44 +0000,nyt,He Was Lost. Could a Shooting Range Like Thund...,https://www.nytimes.com/2025/08/14/us/men-shoo...
4,2025-08-14-18-34-06 +0000,nyt,"Israel Says More Aid Is Entering Gaza, but Rel...",https://www.nytimes.com/2025/08/14/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,48
28,putin,21
599,new,13
21,russia,11
166,police,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
138,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...,116
79,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...,108
279,2025-08-13-20-57-00 +0000,wsj,Trump Agrees on Ukraine Red Lines With Europe ...,https://www.wsj.com/world/trump-talks-ukraine-...,99
2,2025-08-14-18-44-35 +0000,nyt,Russia Seeks to Add Trade and Arms Control Int...,https://www.nytimes.com/2025/08/14/world/europ...,99
22,2025-08-14-17-56-02 +0000,nypost,India shops the globe for oil after 50% tariff...,https://nypost.com/2025/08/14/world-news/india...,98


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
138,116,2025-08-14-11-58-04 +0000,nypost,Russia appears ready to test new missile — eve...,https://nypost.com/2025/08/14/world-news/russi...
62,39,2025-08-14-16-23-09 +0000,nypost,"Toronto film festival reverses course, will ‘e...",https://nypost.com/2025/08/14/world-news/tiff-...
121,37,2025-08-14-13-22-16 +0000,latimes,Hiltzik: Trump's deal with Nvidia puts our nat...,https://www.latimes.com/business/story/2025-08...
58,36,2025-08-14-16-29-11 +0000,nypost,DC cops allowed to alert ICE about illegal imm...,https://nypost.com/2025/08/14/us-news/dc-cops-...
117,34,2025-08-14-13-45-26 +0000,cbc,More than 100 aid groups call on Israel to end...,https://www.cbc.ca/news/world/israel-gaza-aid-...
253,33,2025-08-13-22-21-27 +0000,nypost,Homeless lunatic accused of killing pregnant w...,https://nypost.com/2025/08/13/world-news/homel...
37,32,2025-08-14-17-19-00 +0000,wsj,Mortgage rates edged down to their lowest leve...,https://www.wsj.com/economy/housing/mortgage-r...
5,31,2025-08-14-18-33-53 +0000,nypost,Success Academy Charter students’ test scores ...,https://nypost.com/2025/08/14/us-news/success-...
79,31,2025-08-14-15-59-01 +0000,nypost,Trump reveals the odds of Putin summit ending ...,https://nypost.com/2025/08/14/us-news/trump-gi...
285,30,2025-08-13-20-48-49 +0000,latimes,"They led police on a wild L.A. car chase, then...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
